In [53]:
import numpy as np

import tensorflow as tf

from Fast_SCNN import get_fast_scnn

from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
get_available_gpus()

['/device:GPU:0']

In [4]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [5]:
import pickle
    
def load_pickle(filename):
    
    item_pkl_file = open(filename, 'rb')
    item = pickle.load(item_pkl_file)
    
    return item

## Load Training set

In [6]:
root_dir = '/mnt/windows/projects/data/'
import timeit

In [58]:
images_train = load_pickle(root_dir + "images_train_mini.pickle")
labels_train = load_pickle(root_dir + "labels_train_mini.pickle")


In [59]:
images_utrain = np.asarray(images_train) / 255.0

labels_train = np.asarray(labels_train)

images_utrain = images_utrain.reshape(images_utrain.shape[0],images_utrain.shape[2],images_utrain.shape[1],images_utrain.shape[3])

labels_train = labels_train.reshape(labels_train.shape[0],labels_train.shape[2],labels_train.shape[1])

In [60]:
print(images_utrain.shape, labels_train.shape)


(500, 512, 512, 3) (500, 64, 64)


In [61]:
labels_train_categorical = tf.keras.utils.to_categorical(labels_train, 20)
print(labels_train_categorical.shape)

(500, 64, 64, 20)


In [62]:
image_count = len(images_train)

## Load validation set

In [23]:
images_val = load_pickle(root_dir + "images_val_mini_100.pickle")
labels_val = load_pickle(root_dir + "labels_val_mini_100.pickle")

In [24]:
images_uval = np.asarray(images_val) / 255.0

labels_val = np.asarray(labels_val)

images_uval = images_uval.reshape(images_uval.shape[0],images_uval.shape[2],images_uval.shape[1],images_uval.shape[3])

labels_val = labels_val.reshape(labels_val.shape[0],labels_val.shape[2],labels_val.shape[1])

In [47]:
labels_val_categorical = tf.keras.utils.to_categorical(labels_val, 20)
print(labels_val_categorical.shape)
#print(labels_val_categorical)
image_count_val = len(images_val)

(20, 64, 64, 20)


## Get input

In [63]:
start = timeit.default_timer()

image_ds_train = tf.data.Dataset.from_tensor_slices(images_utrain)
label_ds_train = tf.data.Dataset.from_tensor_slices(labels_train_categorical)

image_ds_val = tf.data.Dataset.from_tensor_slices(images_uval)
label_ds_val = tf.data.Dataset.from_tensor_slices(labels_val_categorical)

stop = timeit.default_timer()
print('Time: ', stop - start) 

Time:  1.7354137339989393


In [64]:
print(image_ds_train, label_ds_train)

<TensorSliceDataset shapes: (512, 512, 3), types: tf.float64> <TensorSliceDataset shapes: (64, 64, 20), types: tf.float32>


In [65]:
image_label_ds = tf.data.Dataset.zip((image_ds_train, label_ds_train))
image_label_val_ds = tf.data.Dataset.zip((image_ds_val, label_ds_val))

In [66]:
print(image_label_ds)
print(image_label_val_ds)
image_count

<ZipDataset shapes: ((512, 512, 3), (64, 64, 20)), types: (tf.float64, tf.float32)>
<ZipDataset shapes: ((512, 512, 3), (64, 64, 20)), types: (tf.float64, tf.float32)>


500

In [67]:
BATCH_SIZE = 8

# Setting a shuffle buffer size as large as the dataset ensures that the data is
# completely shuffled.
ds = image_label_ds.shuffle(buffer_size=image_count)
ds = ds.repeat()
ds = ds.batch(BATCH_SIZE)
# `prefetch` lets the dataset fetch batches, in the background while the model is training.
ds = ds.prefetch(buffer_size=AUTOTUNE)
ds

<PrefetchDataset shapes: ((None, 512, 512, 3), (None, 64, 64, 20)), types: (tf.float64, tf.float32)>

In [68]:
ds = image_label_ds.apply(tf.data.experimental.shuffle_and_repeat(buffer_size = image_count))
ds = ds.batch(BATCH_SIZE)
ds = ds.prefetch(buffer_size=AUTOTUNE)
ds


<PrefetchDataset shapes: ((None, 512, 512, 3), (None, 64, 64, 20)), types: (tf.float64, tf.float32)>

In [48]:
#BATCH_SIZE = 8

# Setting a shuffle buffer size as large as the dataset ensures that the data is
# completely shuffled.
val_ds = image_label_val_ds.shuffle(buffer_size=image_count_val)
val_ds = val_ds.repeat()
val_ds = val_ds.batch(BATCH_SIZE)
# `prefetch` lets the dataset fetch batches, in the background while the model is training.
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
val_ds

<PrefetchDataset shapes: ((None, 512, 512, 3), (None, 64, 64, 20)), types: (tf.float64, tf.float32)>

In [49]:
val_ds = image_label_val_ds.apply(tf.data.experimental.shuffle_and_repeat(buffer_size = image_count_val))
val_ds = val_ds.batch(BATCH_SIZE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
val_ds


<PrefetchDataset shapes: ((None, 512, 512, 3), (None, 64, 64, 20)), types: (tf.float64, tf.float32)>

# Training

In [54]:
fast_scnn = get_fast_scnn(512,512,20)

gfe_layer.shape: (None, 32, 32, 64)
gfe_layer.shape: (None, 16, 16, 96)
gfe_layer.shape: (None, 16, 16, 128)
16 16
x in paramid.shape (None, 16, 16, 128)
x in paramid.shape (None, 16, 16, 128)
x in paramid.shape (None, 16, 16, 128)
x in paramid.shape (None, 16, 16, 128)
gfe_layer.shape: (None, 16, 16, 256)
ff_layer1.shape (None, 64, 64, 128)
ff_layer2.shape (None, 64, 64, 256)
ff_layer2.shape (None, 64, 64, 256)
ff_layer2.shape (None, 64, 64, 128)
ff_final.shape (None, 64, 64, 128)
classifier.shape (None, 64, 64, 128)
classifier.shape (None, 64, 64, 128)
classifier.shape (None, 64, 64, 20)
classifier before upsampling: (None, 64, 64, 20)


### Only tried on limited epochs to make sure the algorithm learns

In [69]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)

model_history = fast_scnn.fit(ds, epochs = 5, validation_data = val_ds, callbacks = [es])

Epoch 1/5
63/63 [==============================] - 257s 4s/step - loss: 1.6742 - accuracy: 0.5493 - val_loss: 2.1127 - val_accuracy: 0.4774
Epoch 2/5
63/63 [==============================] - 257s 4s/step - loss: 1.5743 - accuracy: 0.5785 - val_loss: 1.9813 - val_accuracy: 0.4651
Epoch 3/5
63/63 [==============================] - 258s 4s/step - loss: 1.5174 - accuracy: 0.5987 - val_loss: 1.9836 - val_accuracy: 0.4803
Epoch 4/5
63/63 [==============================] - 264s 4s/step - loss: 1.4712 - accuracy: 0.6110 - val_loss: 1.6498 - val_accuracy: 0.5846
Epoch 5/5
63/63 [==============================] - 257s 4s/step - loss: 1.4365 - accuracy: 0.6195 - val_loss: 1.5724 - val_accuracy: 0.6142
